In [ ]:
import numpy as np
from scipy.integrate import odeint
import sympy as sp
import matplotlib.pyplot as plt

In [ ]:
t,g,m1,m2,l1,l2 = sp.symbols(r"t g m_1 m_2 l_1 l_2")

In [ ]:
theta1, phi1, theta2, phi2 = sp.Function(r'theta_1')(t), sp.Function(r'phi_1')(t), sp.Function(r'theta_2')(t), sp.Function(r'phi_2')(t)

In [ ]:
x1 = l1*sp.sin(theta1)*sp.cos(phi1)
y1 = l1*sp.sin(theta1)*sp.sin(phi1)
z1 = l1*sp.cos(theta1)

x2 = x1 + l2*sp.sin(theta2)*sp.cos(phi2)
y2 = y1 + l2*sp.sin(theta2)*sp.sin(phi2)
z2 = z1 - l2*sp.cos(theta2)

In [ ]:
x1_f = sp.lambdify((theta1, theta2, phi1, phi2, l1, l2), x1)
y1_f = sp.lambdify((theta1, theta2, phi1, phi2, l1, l2), y1)
z1_f = sp.lambdify((theta1, theta2, phi1, phi2, l1, l2), z1)
x2_f = sp.lambdify((theta1, theta2, phi1, phi2, l1, l2), x2)
y2_f = sp.lambdify((theta1, theta2, phi1, phi2, l1, l2), y2)
z2_f = sp.lambdify((theta1, theta2, phi1, phi2, l1, l2), z2)

In [ ]:
theta1d = sp.diff(theta1,t)
phi1d = sp.diff(phi1,t)
theta2d = sp.diff(theta2,t)
phi2d = sp.diff(phi2,t)
theta1dd = sp.diff(theta1d,t)
phi1dd = sp.diff(phi1d,t)
theta2dd = sp.diff(theta2d,t)
phi2dd = sp.diff(phi2d,t)

In [ ]:
# kinetic energy
T = (1/2)*m1*(sp.diff(x1,t)**2 + sp.diff(y1,t)**2 + sp.diff(z1,t)**2) + (1/2)*m2*(sp.diff(x2,t)**2 + sp.diff(y2,t)**2 + sp.diff(z2,t)**2)
# potential energy
V = -m1*g*z1 - m2*g*z2
# lagrangian
L = T - V

In [ ]:
 (1/2)*m2*(sp.diff(x2,t) + sp.diff(y2,t) + sp.diff(z2,t))

Euler-Lagrange Equation::$$\frac{d}{dt}\big(\frac{\partial{L}}{\partial{\dot{q_j}}}\big) - \frac{\partial{L}}{\partial{q_j}} = 0 $$

In [ ]:
Leq1 = sp.diff(sp.diff(L , theta1d),t) - sp.diff(L , theta1)
Leq2 = sp.diff(sp.diff(L , phi1d),t) - sp.diff(L , phi1)
Leq3 = sp.diff(sp.diff(L , theta2d),t) - sp.diff(L , theta2)
Leq4 = sp.diff(sp.diff(L , phi2d), t) - sp.diff(L , phi2)

In [ ]:
sol = sp.solve([Leq1,Leq2,Leq3,Leq4],(theta1dd,phi1dd,theta2dd,phi2dd),simplify=True, relionabbl=False)

In [ ]:
sol

In [ ]:
# sols = []
# for eqns,differentials in zip([Leq1,Leq2,Leq3,Leq4],[theta1dd,phi1dd,theta2dd,phi2dd]):
#     sols.append(sp.solve(eqns,differentials))

* $\:\:\frac{d\theta_1}{dt} = O$
* $\:\:\frac{d^2\theta_1}{dt^2} = \frac{dO}{dt}$
* $\:\:\frac{d\phi_1}{dt} = \omega$
* $\:\:\frac{d^2\phi_1}{dt^2} = \frac{d\omega}{dt}$

In [ ]:
dtheta1dt_f = sp.lambdify(theta1d,theta1d)
dodt_f = sp.lambdify((t,g,m1,m2,l1,l2,theta1,phi1,theta2,phi2,theta1d,phi1d,theta2d,phi2d),sol[theta1dd])
dphi1dt_f = sp.lambdify(phi1d,phi1d)
dwdt_f = sp.lambdify((t,g,m1,m2,l1,l2,theta1,phi1,theta2,phi2,theta1d,phi1d,theta2d,phi2d),sol[phi1dd])
dtheta2dt_f = sp.lambdify(theta2d,sol[theta2d])
dphi2dt_f = sp.lambdify(phi2d,sol[phi2d])

In [ ]:
def dSdt(S,t,g,m1,m2,l1,l2):
    theta1, theta2, phi1, phi2, o, w = S
    return [
        dtheta1dt_f(o),
        dodt_f(t,g,m1,m2,l1,l2,theta1,phi1,theta2,phi2,theta1d,phi1d,theta2d,phi2d),
        dphi1dt_f(w),
        dwdt_f(t,g,m1,m2,l1,l2,theta1,phi1,theta2,phi2,theta1d,phi1d,theta2d,phi2d),
        dtheta2dt_f(theta2),
        dphi2dt_f(phi2)
    ]

In [ ]:
t = np.linspace(0,40,1001)
g = 10
m1 = 2
m2 = 1
l1 = 1
l2 = 1
y0 = [np.pi/3,np.pi/3,0,-np.pi,3,10]
ans = odeint(dSdt,y0,t,args=(g,m1,m2,l1,l2))